In [1]:
import pandas as pd
import numpy as np
import math

from tqdm.notebook import tqdm
import time

In [2]:
path = 'C:/Users/Rusla/Documents/GitHub/Пушкинская карта - стажировка/Sample/'

In [3]:
users = pd.read_csv(path + 'users_full.csv', sep=';', error_bad_lines=False)

In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879681 entries, 0 to 2879680
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_date         int64  
 1   user_id             object 
 2   user_birth          object 
 3   create_date.1       object 
 4   user_phone_details  object 
 5   buyer_mobile_phone  object 
 6   region              object 
 7   age                 int64  
 8   user_region         float64
dtypes: float64(1), int64(2), object(6)
memory usage: 197.7+ MB


In [5]:
users.sample(3)

,create_date,user_id,user_birth,create_date.1,user_phone_details,buyer_mobile_phone,region,age,user_region
54576,1635352133,488ed8e6e6e56d4455123f462d2e779a,2005-01-30,2021-11-15,"Android, 11 (SDK 30)",9ff9fb9984193ec9712d146c7e2a9535,NaN,17,NaN
16918,1635667940,3bed1820cb675773722626dbf57e076d,2002-10-12,2021-11-20,"Android, 11 (SDK 30)",046123c30e48f41aac2ba168a985e39b,NaN,19,NaN
1419752,1630751350,6b380e057bab546e0d416a629413f7b6,2003-11-15,2021-11-19,"Android, 8.0.0 (SDK 26)",d69f94b72d72c18ebcf30852d6f14211,Пермский край,18,59.0


In [6]:
users = users.astype({"age": "object", "user_region": "object"})

In [7]:
import json 
expl = [json.loads(line) for line in open(path + 'tickets2.json', 'r', encoding='utf-8', errors = 'ignore')]

In [8]:
tickets = pd.DataFrame(expl)

In [9]:
tickets.sample(3)

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date
185109,1636874403000,,1265080,1638108000000,Центр Эрмитаж-Урал,9705121040,NaN,8be0e9312fcae3d5225b5c9f5b171483,None,RUB 300.00,Полина,None,1636874427598975974,visited,RUB 300.00,,1.636875e+12
750542,1637502327000,,1299075,1639144800000,Театр-Театр. Большая сцена,9705121040,NaN,8f43dc324139c64e5d07b245e6841733,None,RUB 896.00,Мария,None,1637502349273464413,visited,RUB 896.00,,1.639143e+12
823245,1636826897,,1284691,1637335800,"Партер, ряд 9, место 8",7714836675,NaN,ebd1099e45c6fc2907ad486f97d16eeb,None,550,,None,1636827789155117750,visited,550,6410,1.637334e+09


In [10]:
tickets[tickets['buyer_mobile_phone'].isna()]

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date


In [11]:
tickets.visitor_first_name.nunique() # можно было попробовать пол по именм, но имён у нас всего 11 тысяч из 1,2 млн.

11791

# Добавим данные пользователя в дата сет с купленными билетами

In [12]:
tickets = pd.merge(tickets,users,on=['buyer_mobile_phone'], how='left')

In [13]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854330 entries, 0 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             854330 non-null  int64  
 1   comment                  854330 non-null  object 
 2   session_event_id         854330 non-null  object 
 3   session_date             854330 non-null  int64  
 4   session_params           854330 non-null  object 
 5   owner                    854330 non-null  object 
 6   refund_date              25117 non-null   float64
 7   buyer_mobile_phone       854330 non-null  object 
 8   refund_reason            25117 non-null   object 
 9   payment_amount           854330 non-null  object 
 10  visitor_first_name       854330 non-null  object 
 11  refund_ticket_price      24409 non-null   object 
 12  create_date_x            854330 non-null  int64  
 13  status                   854330 non-null  object 
 14  paym

In [14]:
tickets_2 = tickets[tickets['user_id'].notna() == True]

In [15]:
tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530576 entries, 1 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             530576 non-null  int64  
 1   comment                  530576 non-null  object 
 2   session_event_id         530576 non-null  object 
 3   session_date             530576 non-null  int64  
 4   session_params           530576 non-null  object 
 5   owner                    530576 non-null  object 
 6   refund_date              16455 non-null   float64
 7   buyer_mobile_phone       530576 non-null  object 
 8   refund_reason            16455 non-null   object 
 9   payment_amount           530576 non-null  object 
 10  visitor_first_name       530576 non-null  object 
 11  refund_ticket_price      16025 non-null   object 
 12  create_date_x            530576 non-null  int64  
 13  status                   530576 non-null  object 
 14  paym

In [16]:
cliks_add = pd.read_csv(path + 'cliks_add.csv', sep=';', error_bad_lines=False)

In [17]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5747389 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               float64
 7   session_name             object 
 8   organization_id          float64
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        float64
 12  org_category             object 
 13  age                      float64
 14  user_region              float64
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(5), int64(1), object(11)
memory usage: 745.4+ MB


In [18]:
cliks_add.shape

(5747389, 17)

In [19]:
ticket_users = tickets_2['user_id'].unique()

In [20]:
tickets_2['user_id'].nunique()

329522

In [21]:
len(ticket_users)

329522

In [22]:
cliks_add_users = cliks_add['user_id'].unique()

In [23]:
len(cliks_add_users)

272442

In [24]:
cliks_add['user_id'].nunique()

272442

In [25]:
cliks_add_users[:5]

array(['08014dcd786881f349938c9255e9df36',
       'b2560eaa8bc23dbba5f24aa5075b89ed',
       'fdfe16d86915392aeeca341505cce1c1',
       'cf3f95a25510513018a33990a30f5acb',
       'e2f896fb069e555a5faaf71a3d2b8dec'], dtype=object)

In [26]:
ticket_users[:5]

array(['ef54479d868c93e5b6a6e425fc780954',
       '68f0a5ca2209fedf6d574d74940e55a5',
       '89bc110254468dbf23f51f84385418a0',
       '38d83e61be8f2afd0d335686d685dc56',
       '65835d1c0954fe08db3ffa4e1a5fb75f'], dtype=object)

cliks_and_tickets = cliks_add.copy()

cliks_and_tickets.shape

cliks_and_tickets = pd.DataFrame()

import time
start_time = time.time()

cliks_and_tickets = cliks_add.merge(
    tickets_2.drop_duplicates(subset=['user_id']), left_on='user_id', right_on=['user_id'], how='inner')

print("--- %s seconds ---" % (time.time() - start_time))

cliks_and_tickets.shape

In [27]:
import time
start_time = time.time()

cliks_and_tickets_2 = cliks_add.loc[cliks_add.user_id.isin(tickets_2.user_id.unique())] 

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.485492467880249 seconds ---


In [28]:
cliks_and_tickets_2.shape

(3538545, 17)

In [29]:
cliks_and_tickets_2.user_id.nunique()

137156

In [30]:
#cliks_and_tickets_2.to_csv(path + 'cliks_and_tickets.csv', sep = ';', index = False)

In [31]:
cliks_and_tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538545 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               float64
 7   session_name             object 
 8   organization_id          float64
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        float64
 12  org_category             object 
 13  age                      float64
 14  user_region              float64
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(5), int64(1), object(11)
memory usage: 485.9+ MB


In [32]:
cliks_and_tickets_2 = cliks_and_tickets_2.astype({'session_id': 'object', 'organization_id': 'object', 
                            'Org_region_number': 'object', 'user_region': 'object'}, copy=False)

In [33]:
cliks_and_tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538545 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               object 
 7   session_name             object 
 8   organization_id          object 
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        object 
 12  org_category             object 
 13  age                      float64
 14  user_region              object 
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(1), int64(1), object(15)
memory usage: 485.9+ MB


In [34]:
cliks_and_tickets_2.dropna(subset = ['session_id'], inplace = True)

In [35]:
cliks_and_tickets_2['user_id'] = cliks_and_tickets_2['user_id'].apply(lambda x: str(x))

In [36]:
cliks_and_tickets_2['session_id'] = cliks_and_tickets_2['session_id'].apply(lambda x: str(x))

In [37]:
cliks_and_tickets_2['user_session'] = cliks_and_tickets_2['user_id'] + cliks_and_tickets_2['session_id']

In [38]:
tickets_2['user_session'] = tickets_2['user_id'] + tickets_2['session_event_id']

C:\Users\Rusla\AppData\Local\Temp/ipykernel_16800/3420038588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickets_2['user_session'] = tickets_2['user_id'] + tickets_2['session_event_id']


In [39]:
cliks_and_tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3526081 entries, 0 to 5747388
Data columns (total 18 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               object 
 7   session_name             object 
 8   organization_id          object 
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        object 
 12  org_category             object 
 13  age                      float64
 14  user_region              object 
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
 17  user_session             object 
dtypes: float64(1), int64(1), object(16)
memory usage: 511.1+ MB


In [40]:
tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530576 entries, 1 to 854329
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             530576 non-null  int64  
 1   comment                  530576 non-null  object 
 2   session_event_id         530576 non-null  object 
 3   session_date             530576 non-null  int64  
 4   session_params           530576 non-null  object 
 5   owner                    530576 non-null  object 
 6   refund_date              16455 non-null   float64
 7   buyer_mobile_phone       530576 non-null  object 
 8   refund_reason            16455 non-null   object 
 9   payment_amount           530576 non-null  object 
 10  visitor_first_name       530576 non-null  object 
 11  refund_ticket_price      16025 non-null   object 
 12  create_date_x            530576 non-null  int64  
 13  status                   530576 non-null  object 
 14  paym

In [41]:
cliks_and_tickets_3 = cliks_and_tickets_2.merge(tickets_2[['user_session', 'payment_date', 'session_date', 'payment_amount',
                                                          'status', 'payment_ticket_price']], 
                                                on='user_session', how='left')

In [42]:
cliks_and_tickets_3.describe()

,age,user_phone_details_id_2,payment_date,session_date
count,3.526081e+06,3.526081e+06,0.0,0.0
mean,1.838982e+01,1.327668e+02,NaN,NaN
std,2.535062e+00,6.778106e+01,NaN,NaN
min,1.400000e+01,1.500000e+01,NaN,NaN
25%,1.600000e+01,1.100000e+02,NaN,NaN
50%,1.800000e+01,1.110000e+02,NaN,NaN
75%,2.100000e+01,2.140000e+02,NaN,NaN
max,2.300000e+01,2.150000e+02,NaN,NaN


In [43]:
cliks_and_tickets_3['payment_date'].value_counts()

Series([], Name: payment_date, dtype: int64)